In [2]:
from tensorflow.keras.models import load_model
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt 
from config import *

CATEGORIES_REVERSE = {v: k for k, v in CATEGORIES_CODES.items()}

In [7]:
class PredictWithModel:
    shape = (224, 224, 1)
    
    def __init__(self, version: str) -> None:
        self.test_img_path = 'data/test'
        self.model = load_model(f'models/model_v{version}.h5')
        self.path = os.listdir(self.test_img_path)
        self.images = np.empty((_s:=len(self.path), ) + self.shape, dtype='float16')
        self.predictions = pd.DataFrame(columns=['ID', 'Target'])
    
    def parse_img(self, img_path: str) -> np.array:
        img = ~cv2.imread(f"{self.test_img_path}/{img_path}", 0) / 255
        return np.asarray(img, dtype='float16').reshape(self.shape)
    
    def predict_img(self):
        for i, img_path in enumerate(self.path):
            self.images[i] = self.parse_img(img_path)
            
        for i, p in enumerate(self.model.predict(self.images)):
            self.predictions.loc[i] = [self.path[i], CATEGORIES_REVERSE[np.argmax(p)]]
        self.predictions.to_csv('data/result/predictions.csv', index=False)
            
    @staticmethod
    def show(img: np.array) -> None:
        plt.imshow(img[0], cmap='gray')


In [8]:
model_version = '0.1.1'
predict = PredictWithModel(model_version)
predict.predict_img()

161/161 [==============================] - 19s 116ms/step
